### 라이브러리 불러오기

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import joblib


### 파일 불러오기

In [33]:
df = pd.read_csv('./finalData/Yeosu_05.csv', na_values='-', encoding='cp949')
df = df.replace('-', np.nan)

# 위 값으로 채우기
df = df.fillna(method='ffill')

# 맨 앞도 채워야 한다면 아래 값으로 채우기
df = df.fillna(method='bfill')
df.to_csv('Yeosu_05.csv', index=False)

C:\Users\kosmo\AppData\Local\Temp\ipykernel_3148\319022664.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')
C:\Users\kosmo\AppData\Local\Temp\ipykernel_3148\319022664.py:8: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='bfill')


## 빈 셀은 윗 값으로 채우고 조건에 따른 ['can_sail'] 컬럼 만들기

In [11]:
df = pd.read_csv('./finalData/Yeosu_06.csv', na_values='-', encoding='cp949')
# df = pd.read_csv('./finalData/Taean_06.csv', na_values='-'), encoding='cp949'

df = df.replace('-', np.nan)

# 위 값으로 채우기
df = df.fillna(method='ffill')

# 맨 앞도 채워야 한다면 아래 값으로 채우기
df = df.fillna(method='bfill')

print(df.isnull().sum()) 
def label_can_sail(row):
    # 풍속
    if row['wind_speed'] > 8:
        return 0  # 못 뜸
    # 기압
    if row['pressure'] < 990 or row['pressure'] > 1030 :
        return 0

    if row['sea_speed'] > 70:
        return 0
    
    if row['sea_high'] < 80:
        return 0
    # 예시: 다 괜찮으면
    return 1

df['can_sail'] = df.apply(label_can_sail, axis=1)
print(df['can_sail'].value_counts())

df.to_csv('./finalData/Yeosu_06.csv', index=False)

datetime      0
sea_high      0
salt          0
wind_speed    0
wind_dir      0
temp          0
pressure      0
sea_speed     0
sea_dir_s     0
sea_dir_i     0
sea_temp      0
dtype: int64
can_sail
1    4029
0     291
Name: count, dtype: int64


C:\Users\kosmo\AppData\Local\Temp\ipykernel_8608\3401155766.py:7: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')
C:\Users\kosmo\AppData\Local\Temp\ipykernel_8608\3401155766.py:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='bfill')


## 훈련 및 테스트 데이터 불러오기

In [56]:
df_train = pd.read_csv('./finalData/Yeosu_04.csv', na_values='-')
df_test = pd.read_csv('./finalData/Yeosu_05.csv', na_values='-')

## 데이터 전처리 및 모델 생성

In [ ]:

# -------------------------------
# 2️⃣ 피처 & 타겟 정의
# -------------------------------
features = ['wind_speed', 'pressure', 'sea_speed', 'sea_high']
target_col = 'can_sail'  # 실제 운항 가능 여부 (0/1)

X_train = df_train[features].dropna()
y_train = df_train[target_col].dropna()

X_test = df_test[features].dropna()
y_test = df_test[target_col].dropna()

# -------------------------------
# 3️⃣ 스케일링
# -------------------------------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# -------------------------------
# 4️⃣ MLP 모델 개선 버전
# -------------------------------
model = Sequential()
model.add(Dense(128, input_shape=(X_train_scaled.shape[1],)))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(1, activation='sigmoid'))  # 이진 분류

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# model.summary()

# -------------------------------
# 5️⃣ 콜백 설정
# -------------------------------
es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
rlr = ReduceLROnPlateau(monitor='val_loss', patience=5, factor=0.5, verbose=1)




## 학습 및 테스트하기
### 모델 & 스케일러 저장

In [ ]:
# -------------------------------
# 6️⃣ 학습
# -------------------------------
history = model.fit(
    X_train_scaled, y_train,
    validation_data=(X_test_scaled, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[es, rlr]
)

# -------------------------------
# 7️⃣ 테스트 데이터 평가
# -------------------------------
y_test_pred = (model.predict(X_test_scaled) > 0.5).astype(int)
print("\n=== Test Classification Report ===")
print(classification_report(y_test, y_test_pred))

# -------------------------------
# 8️⃣ 모델 & 스케일러 저장
# -------------------------------
model.save('Yeosu_can_sail_model.h5')
joblib.dump(scaler, 'Yeosu_scaler.pkl')

print("\n✅ MLP 모델 저장 완료")
print("✅ 스케일러 저장 완료")


## 6월 데이터 예측하기
### can_sail 예측 모델 사용

In [ ]:
# -------------------------------
# 🔮 9️⃣ 나중에 InCheon_06.csv 들어올 때 예측 흐름 예시
# -------------------------------

# ✅ 나중에 새 데이터 들어오면 이렇게 쓰면 돼요!

import pandas as pd
import joblib
from tensorflow.keras.models import load_model

# 빈 DataFrame 생성
col = ['wind_speed', 'pressure', 'sea_speed', 'sea_high']
df_empty = pd.DataFrame(columns=col)

# CSV로 저장
df_empty.to_csv('./pred_file/InCheon_06.csv', index=False)

print("✅ 빈 CSV 파일 생성 완료: empty_file.csv")

# 새 데이터 불러오기
df_new = pd.read_csv('./pred_file/InCheon_06.csv')

# 피처 정의
features = ['tide', 'salinity', 'water_temp', 'wind_speed', 'wind_dir', 'air_temp', 'pressure']

# 스케일러 불러오기 & transform
scaler = joblib.load('./model/InCheon_scaler.pkl')
X_new_scaled = scaler.transform(df_new[features])

# 모델 불러오기 & 예측
model = load_model('./model/InCheon_can_sail_model.h5')
y_new_pred = (model.predict(X_new_scaled) > 0.5).astype(int)

df_new['can_sail_pred'] = y_new_pred
df_new.to_csv('InCheon_06_with_MLP_pred.csv', index=False)
print('✅ 06 데이터 예측 결과 저장 완료!')



In [ ]:
import pandas as pd
import joblib
from tensorflow.keras.models import load_model

# ----------------------------------------
# ✅ 1) 저장된 스케일러와 모델 불러오기
# ----------------------------------------
scaler = joblib.load('./model/InCheon_scaler.pkl')
model = load_model('./model/InCheon_can_sail_model.h5')

# ----------------------------------------
# ✅ 2) 04.csv or 05.csv 불러오기 (가상 6월 데이터)
# ----------------------------------------
# 예: 05.csv를 6월이라고 가정
df_fake_06 = pd.read_csv('./pred_file/InCheon_06.csv')

# ----------------------------------------
# ✅ 3) 학습 때 사용한 피처만 선택
# ----------------------------------------
features = ['wind_speed', 'pressure', 'sea_speed', 'sea_high']

X_fake_06_scaled = scaler.transform(df_fake_06[features])

# ----------------------------------------
# ✅ 4) 예측
# ----------------------------------------
y_fake_06_pred = (model.predict(X_fake_06_scaled) > 0.5).astype(int)

# ----------------------------------------
# ✅ 5) 결과 붙이고 새 파일로 저장
# ----------------------------------------
df_fake_06['can_sail_pred'] = y_fake_06_pred
df_fake_06.to_csv('./pred_file/InCheon_06.csv', index=False)

print('✅ (가상) 06 데이터 예측 결과 저장 완료: InCheon_06_with_MLP_pred.csv')
